In [1]:
from common import *
import smart_open

In [2]:
with open(join(DATA_FOLDER, 'gold_mongo.json'), 'r') as f:
    gold = json.load(f)

In [10]:
def iter_docs(fnames):
    for i, fn in enumerate(fnames):
#         logging.info("%s: " % i + fn)
        with GzipFile(fn, 'rb') as f:
            docs = ujson.loads(f.read())
        for k,doc in docs.items():
            if k not in gold.keys():
                yield k,doc

                
class Documents(object):
    def __init__(self, folder, tokens_only=False):
        self.folder = folder
        self.tokens_only = tokens_only
 
    def __iter__(self):
        fnames = glob(join(self.folder, '*.json.gz'))
        for k, doc in iter_docs(fnames):
            unlisted = [w for s in doc for w in s]
            if self.tokens_only:
                yield unlisted
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(unlisted, [k])

In [ ]:
folder = join(DATA_FOLDER, 'documents/')

model = Doc2Vec(Documents(folder), size=200, window=8, min_count=5, workers=cpu_count, iter=10)

2017-09-28 22:04:31,016 [MainThread  ] [DEBUG]  Fast version of gensim.models.word2vec is being used
2017-09-28 22:04:31,017 [MainThread  ] [INFO ]  collecting all words and their counts
2017-09-28 22:04:31,318 [MainThread  ] [INFO ]  PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-09-28 22:04:48,899 [MainThread  ] [INFO ]  PROGRESS: at example #10000, processed 18251055 words (1038695/s), 156491 word types, 10000 tags
2017-09-28 22:05:03,626 [MainThread  ] [INFO ]  PROGRESS: at example #20000, processed 32653756 words (978083/s), 248872 word types, 20000 tags
2017-09-28 22:05:13,143 [MainThread  ] [INFO ]  PROGRESS: at example #30000, processed 41525327 words (932271/s), 298791 word types, 30000 tags
2017-09-28 22:05:27,058 [MainThread  ] [INFO ]  PROGRESS: at example #40000, processed 54836204 words (956715/s), 358164 word types, 40000 tags
2017-09-28 22:05:42,074 [MainThread  ] [INFO ]  PROGRESS: at example #50000, processed 69788323 words (995855/s), 432

In [ ]:
model.save('../data/saved/d2v_model_s200_w8_mc5_iter10')